In [2]:
%matplotlib notebook
import numpy as np
from numpy.fft import fft2, ifft2, fftshift
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import scipy
from scipy.ndimage import gaussian_filter1d as gf1d
from scipy.ndimage import gaussian_filter as gf
from scipy.ndimage import uniform_filter as uf

import sys
sys.path.append("../kai_colloids/PyDDM") #must point to the PyDDM folder
import ddm_analysis_and_fitting as ddm

import tiff_file
import ddm_clean 

import io 
import sys
import csv
import os
import glob #glob is helpful for searching for filenames or directories
import pickle #for saving data

In [2]:
def last_3chars(x):
    return((x.split('\\')[-1])[-7:-4])

In [3]:
###################################################################################
# Specify where the data is and file name for image (or image sequence)           #
###################################################################################

directory = "Z"
exp = "6-27-22_1-3_vs_2-1"
data_dir = directory+":\\Gregor L\\__Kai Colloids\\"+exp+"\\all tiff files\\" 
data_dir2 = directory+":\\Gregor L\\__Kai Colloids\\"+exp+"\\reordered tiffs\\" #bottom_row_t01\\
plot_saveto= directory+":\\Gregor L\\__Kai Colloids\\SIA\\Data\\"+exp+"\\"

files = glob.glob(data_dir+"*_t*")
#files = sorted(files, key = last_3chars)
print("found %i files" % len(files))
print(int((len(files))/3))
for i,f in enumerate(files): print (' %i \t %s' % (i, f.split('\\')[-1]))

found 28 files
9
 0 	 AE_40x_t1.tif
 1 	 AE_40x_t2.tif
 2 	 AE_40x_t3.tif
 3 	 AE_40x_t4.tif
 4 	 AE_40x_t5.tif
 5 	 AE_40x_t6.tif
 6 	 AE_40x_t7.tif
 7 	 WT_1-3_40x_t1.tif
 8 	 WT_1-3_40x_t2.tif
 9 	 WT_1-3_40x_t3.tif
 10 	 WT_1-3_40x_t4.tif
 11 	 WT_1-3_40x_t5.tif
 12 	 WT_1-3_40x_t6.tif
 13 	 WT_1-3_40x_t7.tif
 14 	 WT_2-1_40x_t1.tif
 15 	 WT_2-1_40x_t2.tif
 16 	 WT_2-1_40x_t3.tif
 17 	 WT_2-1_40x_t4.tif
 18 	 WT_2-1_40x_t5.tif
 19 	 WT_2-1_40x_t6.tif
 20 	 WT_2-1_40x_t7.tif
 21 	 WT_nokaiA_40x_t1.tif
 22 	 WT_nokaiA_40x_t2.tif
 23 	 WT_nokaiA_40x_t3.tif
 24 	 WT_nokaiA_40x_t4.tif
 25 	 WT_nokaiA_40x_t5.tif
 26 	 WT_nokaiA_40x_t6.tif
 27 	 WT_nokaiA_40x_t7.tif


In [7]:
###### # specify each condition (frame_names) and the time points of data collection (time_array) #
############################################################################################
frame_names = ["1-3 kA-KaiC", "WT KaiC (-kA)", "2-1 kA-KaiC", "AE KaiC (-kA)"]
#["EE KaiC (-kA)", "EA KaiC (-kA)", "WT KaiC (-kA)", "AE KaiC (-kA)"]
#["50% bKaiB", "35% bKaiB", "20% bKaiB", "AE (20% bKaiB)"]
#["1-3 kA-WT", "WT (no kA)", "EA KaiC", "AE KaiC"]
time_array =  [0.42, 4.75, 8.75, 12.75, 19.92, 22.92, 25.42]
#[0.5, 4.5, 8, 12, 20, 24]
#[0.5, 3.5, 6.5, 9.3, 12.1, 15, 18, 21, 24.5, 27.3, 40.5, 44.3]  
#[0.5, 3.2, 17.75, 20, 22, 24.25, 26, 41.75, 44]
#[0.85, 3.5, 18, 20.4, 22.25, 24.5, 26.2, 41.75, 44.2]
#s1 [0.67, 3.67, 6.67, 10, 12.83, 17.67, 21.67, 24.92, 28.17, 45.67, 49.08, 69.25]
#s2 [1, 3.83, 6.83, 10.17, 13.08, 18, 21.92, 25.25, 28.67, 45.92, 49.33, 69.42]
fig_size = 10,10/1.618
font_size = 16
dpi_num = 800
pixel_size = 0.364 # 4*0.091 = 0.364

print((files[0]))
eg_im= tiff_file.imread(files[0])
print(eg_im.shape)
frames_per_tiff = int(eg_im.shape[0])
print(frames_per_tiff)
arr_length = int((len(files))/4)
print(arr_length)

Z:\Gregor L\__Kai Colloids\6-27-22_1-3_vs_2-1\all tiff files\AE_40x_t1.tif
(5, 1440, 1920)
5
7


In [12]:
#arr_length = int((len(files))/4)
t_array = ["_t1", "_t2", "_t3", "_t4", "_t5", "_t6", "_t7"]
frame1_testim = tiff_file.imread(files[0],key=[0])
row1 = np.zeros((5, frame1_testim.shape[0], frame1_testim.shape[1]), dtype=np.uint16)
row2 = np.zeros((5, frame1_testim.shape[0], frame1_testim.shape[1]), dtype=np.uint16)
row3 = np.zeros((5, frame1_testim.shape[0], frame1_testim.shape[1]), dtype=np.uint16)
print(row3)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [13]:
###################################################################################################
# makes new tiff files, each new tiff has all consecutive time points for each specific condition #
###################################################################################################


for i in range(arr_length):

    for j in range(3):     
        frame1_im = tiff_file.imread(files[i],key=[j])
        frame2_im = tiff_file.imread(files[i+arr_length],key=[j])
        frame3_im = tiff_file.imread(files[i+(arr_length*2)],key=[j])
        frame4_im = tiff_file.imread(files[i+(arr_length*3)],key=[j])
        #print(br_frame1_im.shape[0])
    
        if j == 0:
            row1[0] = frame2_im  #WT 1-3
            row1[1] = frame4_im  #WT no kaiA
            row1[2] = frame3_im  #WT 2-1
            row1[3] = frame1_im  #AE
        if j == 1:
            row2[0] = frame2_im
            row2[1] = frame4_im
            row2[2] = frame3_im
            row2[3] = frame1_im
        if j == 2:
            row3[0] = frame2_im
            row3[1] = frame4_im
            row3[2] = frame3_im
            row3[3] = frame1_im

    tiff_file.imsave(data_dir2 +'row1_'+t_array[i]+'.tif', row1)
    tiff_file.imsave(data_dir2 +'row2_'+t_array[i]+'.tif', row2)
    tiff_file.imsave(data_dir2 +'row3_'+t_array[i]+'.tif', row3)
    print(row1.shape)

(5, 1440, 1920)
(5, 1440, 1920)
(5, 1440, 1920)
(5, 1440, 1920)
(5, 1440, 1920)
(5, 1440, 1920)
(5, 1440, 1920)


In [28]:
#####################################################
# **optional** preview of tiff files to be analyzed #
#####################################################
tiff_name = 'row3_'+frame_names[0]
tiff_to_show = tiff_file.imread(data_dir2 +tiff_name+'.tif')

fig = plt.figure(figsize=(10,10))    
for i in range (arr_length):
    ax = plt.subplot(4,4,i+1) ##change according to number of frames (6 --> 3,3; 12 --> 4,4)
    ax.set_title("~" + str(time_array[i]) + " hrs", fontsize=10)
    ax.matshow(tiff_to_show[i], cmap = 'gray') #fig.set_cmap('hot') 
    plt.axis('off')
    plt.tight_layout(pad=.25)
fig.savefig(data_dir2+tiff_name+".jpg", dpi=(dpi_num))

TypeError: len() takes exactly one argument (0 given)